# Elexon API Caller

In this notebook a class is made which creates a Python wrapper for the Elexon BMRS API

The BMRS API Guide can be found <a href="https://www.elexon.co.uk/documents/training-guidance/bsc-guidance-notes/bmrs-api-and-data-push-user-guide-2/">here</a>.

<br>

### Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, date

import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict
import xmltodict

import requests
from urllib import parse

<br>

### Generator Data API Access

In [8]:
## BMRS API Data Stream Caller

class Caller(object):  
    """
    This is an API caller object for the BMRS data streams.
    
    Given an API key and data stream ID the Caller is initialized.
    The Caller's query function is then called with the relevant kwargs for the data stream.
    The url is constructed using the kwargs and the request made, returning the response
    
    """
    
    ## To Do: Account for days where the clocks change (max SPs of 46 and 50)
    
    
    ## ~~~~~~~~~~~~ Core Functions ~~~~~~~~~~~~
        
    def create_url(self, *args, **kwargs):
        ## Creating a mapper for the url template using the kwargs passed in to the query function and the API key
        url_mapper = locals()['kwargs']
        url_mapper['api_key'] = self.key
            
        ## Mapping the kwargs and key onto the url template from the 'stream_url_dict'
        url = self.stream_url_dict[self.stream].format_map(url_mapper)
        
        return url
        
    def make_call(self, url):
        header = { 'Accept': 'application/xml' }
        response = requests.get(url, headers=header)
        
        return response
    
    def query(self, *args, **kwargs):
        url = self.create_url(*args, **kwargs)
        response = self.make_call(url)
        
        self.latest_response = response
        return response
          
        
    ## ~~~~~~~~~~~~ Initializer & Helper Functions ~~~~~~~~~~~~
    
    def info(self):
        info_dict = {
            'key' : self.key,
            'stream' : self.stream
        }
        
        return info_dict
    
    def __init__(self, key, stream):
        self.key = key
        self.stream = stream
    
        
    ## ~~~~~~~~~~~~ Data Objects ~~~~~~~~~~~~
    
    ## Dictionary containing the urls for the different data streams
    stream_url_dict = {

        'B1610' : '\
        https://api.bmreports.com/BMRS/B1610/v1?\
        APIKey={api_key}&\
        SettlementDate={query_date}&\
        Period={SP}&\
        NGCBMUnitID={unit_id}&\
        ServiceType=xml',

        'FUELHH' : '\
        https://api.bmreports.com/BMRS/FUELHH/v1?\
        APIKey={api_key}&\
        FromDate={start_date}&\
        ToDate={end_date}&\
        ServiceType=xml',

        'BOD' : '\
        https://api.bmreports.com/BMRS/BOD/v1?\
        APIKey={api_key}&\
        SettlementDate={query_date}&\
        SettlementPeriod={SP}&\
        BMUnitId={unit_id}&\
        ServiceType=xml',
        
        'ImbalancePricesService' : '\
        https://api.bmreports.com/BMRS/B1770/v1?\
        APIKey={api_key}&\
        SettlementDate={query_date}&\
        Period={SP}&\
        ServiceType=xml',
    } 
    
class API2DataFrameParser(Caller):
        
    def call_2_df(self, *args, **kwargs):
        response = self.query(self, *args, **kwargs) # Run the query, response is saved
        df = self.response_2_df(response) # Converts the response to a dataframe
    
        ## Checking that comb_df attribute exists and initialising it if not
        ## Change it so that the df is initialised at the start and has a function to be reset
        self.single_df = df
        
        return df
    
    def check_response(self, response):
        ## Checking http code in response
        
        http_code = xmltodict.parse(response.text)['response']['responseMetadata']['httpCode']
        
        def f_pass():
            return 'None'
        def f_return_blank_df():
            return pd.DataFrame()
        
        http_code_response_dict = {
            '200' : f_pass(),
            '204' : f_return_blank_df(),
        }
        
        response_2_API_response = http_code_response_dict[http_code]
        return response_2_API_response
    
    def response_2_df(self, response):
        
        response_2_API_response = self.check_response(response)
    
        if not isinstance(response_2_API_response, str):
            return response_2_API_response
        
        ## Converting response to df
        r_dict = xmltodict.parse(response.text)['response']['responseBody']['responseList']['item']

        if isinstance(r_dict, OrderedDict):
            df = pd.DataFrame(r_dict, index=[1])
        else:
            df = pd.DataFrame(r_dict)

        return df
    
class SP_Aggregator(API2DataFrameParser):
    
    def date_range_to_dates(self, start_date, end_date):
        # Accepts two date objects and returns the days between them as a list of strings
        # For the API, dates must be formatted as a string of the form'YYYY-MM-DD'    

        # Overall timedelta is determined and empty results list made
        delta = end_date - start_date
        settlement_dates = []

        # Number of days extracted from timedelta, each day is cycled over and a date string created before appending to results list
        for i in range(delta.days + 1):
            iter_date = start_date + timedelta(i)
            date_string = datetime.strftime(iter_date, '%Y-%m-%d')
            settlement_dates.append(date_string)

        return settlement_dates
    
    def date_range_to_df(self, start_date, end_date, **kwargs):
        
        ## *** Also add in list of days with 46 and 50 SPs ***
        
        settlement_dates = self.date_range_to_dates(start_date, end_date) # Returns the days between them as a list of strings
        norm_SP_list = list(map(str, list(range(1, 49))))
        comb_df = pd.DataFrame()

        for settlement_date in settlement_dates:
            for SP in norm_SP_list:
                
                try:
                    df_gen_SP = self.call_2_df(SP=SP, query_date=settlement_date, **kwargs) # Temp dataframe is made from json at url
                except:
                    raise APICallError(SP, settlement_date)
                    
                if df_gen_SP is not None:
                    comb_df = comb_df.append(df_gen_SP) # Temp dataframe is appended to the main dataframe
                    
        self.comb_df = comb_df
        
        return comb_df
    
class APICallError(Exception):
    def __init__(self, SP, date):
        print(f'API Call failed at SP: {SP}, Date: {date}')

<br>

### User Key

Assign your Elexon API key, if you don't have one register <a href="https://www.elexonportal.co.uk/registration/newuser">here</a>.

In [ ]:
API_key = 'bcz568ze0fnmog3'

<br>

### Half Hourly Generation by Fuel Type

In [3]:
start_date = '2017-10-01'
end_date = '2017-10-07'

FUELHH_aggregator = API2DataFrameParser(API_key, 'FUELHH')
FUELHH_aggregator.call_2_df(start_date=start_date, end_date=end_date)
FUELHH_aggregator.single_df.head(10)

,recordType,startTimeOfHalfHrPeriod,settlementPeriod,ccgt,oil,coal,nuclear,wind,ps,npshyd,ocgt,other,intfr,intirl,intned,intew,biomass,intnem,activeFlag
0,FUELHH,2017-10-01,1,9714,0,0,7389,2166,0,393,0,836,1404,274,724,424,0,0,Y
1,FUELHH,2017-10-01,2,8930,0,0,7418,2259,0,365,0,833,1404,238,714,430,0,0,Y
2,FUELHH,2017-10-01,3,8070,0,0,7441,2461,0,343,0,830,1402,284,708,492,0,0,Y
3,FUELHH,2017-10-01,4,7938,0,0,7463,2537,0,340,0,831,1404,288,732,502,0,0,Y
4,FUELHH,2017-10-01,5,7643,0,0,7468,2670,0,338,0,830,1404,288,696,400,0,0,Y
5,FUELHH,2017-10-01,6,7406,0,0,7468,2837,0,337,0,833,1404,258,706,346,0,0,Y
6,FUELHH,2017-10-01,7,6842,0,0,7474,3106,0,336,0,834,1404,184,680,426,0,0,Y
7,FUELHH,2017-10-01,8,6272,0,0,7469,3433,0,330,0,834,1404,214,674,432,0,0,Y
8,FUELHH,2017-10-01,9,5781,0,0,7470,3554,0,327,0,812,1404,270,710,432,0,0,Y
9,FUELHH,2017-10-01,10,5189,0,0,7479,3791,0,328,0,832,1404,290,686,432,0,0,Y


<br>

### Bid Offer Prices

In [4]:
start_date = date(2018, 8, 1)
end_date = date(2018, 8, 2)

imb_price_aggregator = SP_Aggregator(API_key, 'ImbalancePricesService')
imb_price_aggregator.date_range_to_df(start_date, end_date)

df_imb_price_raw = imb_price_aggregator.comb_df.copy()
df_imb_price_raw.head()

,timeSeriesID,businessType,controlArea,settlementDate,settlementPeriod,imbalancePriceAmountGBP,priceCategory,curveType,resolution,documentType,processType,activeFlag,docStatus,documentID,documentRevNum
0,ELX-EMFIP-IMBP-TS-1,Balance energy deviation,10YGB----------A,2018-08-01,1,39.16771,Insufficient balance,Sequential fixed size block,PT30M,Imbalance prices,Realised,Y,Final,ELX-EMFIP-IMBP-19140316,1
1,ELX-EMFIP-IMBP-TS-2,Balance energy deviation,10YGB----------A,2018-08-01,1,39.16771,Excess balance,Sequential fixed size block,PT30M,Imbalance prices,Realised,Y,Final,ELX-EMFIP-IMBP-19140316,1
0,ELX-EMFIP-IMBP-TS-1,Balance energy deviation,10YGB----------A,2018-08-01,2,45.05,Insufficient balance,Sequential fixed size block,PT30M,Imbalance prices,Realised,Y,Final,ELX-EMFIP-IMBP-19140473,1
1,ELX-EMFIP-IMBP-TS-2,Balance energy deviation,10YGB----------A,2018-08-01,2,45.05,Excess balance,Sequential fixed size block,PT30M,Imbalance prices,Realised,Y,Final,ELX-EMFIP-IMBP-19140473,1
0,ELX-EMFIP-IMBP-TS-1,Balance energy deviation,10YGB----------A,2018-08-01,3,37.1,Insufficient balance,Sequential fixed size block,PT30M,Imbalance prices,Realised,Y,Final,ELX-EMFIP-IMBP-19140638,1


<br>

### Bid Offer Data Retrieval

In [5]:
start_date = date(2018, 8, 1)
end_date = date(2018, 8, 2)

BOD_aggregator = SP_Aggregator(API_key, 'BOD')
BOD_aggregator.date_range_to_df(start_date, end_date, unit_id='*')

df_bod_raw = BOD_aggregator.comb_df.copy()
df_bod_raw.head()

,recordType,bmUnitID,bMUnitType,leadPartyName,ngcBMUnitName,settlementDate,settlementPeriod,bidOfferPairNumber,timeFrom,bidOfferLevelFrom,timeTo,bidOfferLevelTo,bidPrice,offerPrice,activeFlag
0,BOD,2__ABGAS000,G,British Gas Trading Ltd,EAS-BGS01,2018-08-01,1,1,2018-07-31 23:00:00,0,2018-07-31 23:30:00,0,0.00,0.00,Y
1,BOD,2__ABGAS000,G,British Gas Trading Ltd,EAS-BGS01,2018-08-01,1,-1,2018-07-31 23:00:00,0,2018-07-31 23:30:00,0,0.00,0.00,Y
2,BOD,2__AEDIR000,G,British Gas Trading Ltd,EAS-EDR01,2018-08-01,1,1,2018-07-31 23:00:00,0,2018-07-31 23:30:00,0,0.00,0.00,Y
3,BOD,2__AEDIR000,G,British Gas Trading Ltd,EAS-EDR01,2018-08-01,1,-1,2018-07-31 23:00:00,0,2018-07-31 23:30:00,0,0.00,0.00,Y
4,BOD,2__AEELC000,G,E.ON Energy Solutions Limited,EAS-EAS01,2018-08-01,1,1,2018-07-31 23:00:00,0,2018-07-31 23:30:00,0,0.00,0.00,Y


<br>

### Generator Data Retrieval

In [5]:
start_date = date(2018, 8, 1)
end_date = date(2018, 8, 7)

B1610_aggregator = SP_Aggregator(API_key, 'B1610')
B1610_aggregator.date_range_to_df(start_date, end_date, unit_id='*')

df_gen_raw = B1610_aggregator.comb_df.copy()
df_gen_raw.head(10)

,documentType,businessType,processType,timeSeriesID,quantity,curveType,resolution,settlementDate,settlementPeriod,powerSystemResourceType,registeredResourceEICCode,marketGenerationUnitEICCode,marketGenerationBMUId,marketGenerationNGCBMUId,bMUnitID,nGCBMUnitID,activeFlag,documentID,documentRevNum
0,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-227,22.08,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W0000GYMRO-17K,48W0000GYMRO-17K,T_GYMR-17,GYMRO-17,T_GYMR-17,GYMRO-17,Y,ELX-EMFIP-AGOG-19178240,1
1,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-228,16.34,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W0000GYMRO-26J,48W0000GYMRO-26J,T_GYMR-26,GYMRO-26,T_GYMR-26,GYMRO-26,Y,ELX-EMFIP-AGOG-19178240,1
2,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-229,13.5,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W0000GYMRO-28F,48W0000GYMRO-28F,T_GYMR-28,GYMRO-28,T_GYMR-28,GYMRO-28,Y,ELX-EMFIP-AGOG-19178240,1
3,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-153,79.82,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W00000LNMTH-2P,48W00000LNMTH-2P,E_LYNE2,LNMTH-2,E_LYNE2,LNMTH-2,Y,ELX-EMFIP-AGOG-19178240,1
4,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-154,76.04,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W00000LNMTH-3N,48W00000LNMTH-3N,E_LYNE3,LNMTH-3,E_LYNE3,LNMTH-3,Y,ELX-EMFIP-AGOG-19178240,1
5,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-282,110.808,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W00000WDNSO-2F,48W00000WDNSO-2F,T_WDNSO-2,WDNSO-2,T_WDNSO-2,WDNSO-2,Y,ELX-EMFIP-AGOG-19178240,1
6,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-281,111.228,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W00000WDNSO-1H,48W00000WDNSO-1H,T_WDNSO-1,WDNSO-1,T_WDNSO-1,WDNSO-1,Y,ELX-EMFIP-AGOG-19178240,1
7,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-184,84.556,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W00000BRBEO-17,48W00000BRBEO-17,T_BRBEO-1,BRBEO-1,T_BRBEO-1,BRBEO-1,Y,ELX-EMFIP-AGOG-19178240,1
8,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-256,48.326,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W00000RCBKO-2Q,48W00000RCBKO-2Q,T_RCBKO-2,RCBKO-2,T_RCBKO-2,RCBKO-2,Y,ELX-EMFIP-AGOG-19178240,1
9,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-203,645.188,Sequential fixed size block,PT30M,2018-08-01,1,Generation,48W00000DRAXX-1E,48W00000DRAXX-1E,T_DRAXX-1,DRAXX-1,T_DRAXX-1,DRAXX-1,Y,ELX-EMFIP-AGOG-19178240,1


In [6]:
s_gen_unique = df_gen_raw.apply(pd.Series.nunique)     
non_uniq_to_drop = list(s_gen_unique[s_gen_unique==1].index)
cols_to_drop = non_uniq_to_drop + ['timeSeriesID', 'registeredResourceEICCode', 'marketGenerationUnitEICCode', 'marketGenerationBMUId', 'marketGenerationNGCBMUId', 'bMUnitID', 'documentID']
df_gen_trim = df_gen_raw.drop(columns=cols_to_drop)
df_gen_trim.head(10)

,quantity,settlementDate,settlementPeriod,nGCBMUnitID
0,22.08,2018-08-01,1,GYMRO-17
1,16.34,2018-08-01,1,GYMRO-26
2,13.5,2018-08-01,1,GYMRO-28
3,79.82,2018-08-01,1,LNMTH-2
4,76.04,2018-08-01,1,LNMTH-3
5,110.808,2018-08-01,1,WDNSO-2
6,111.228,2018-08-01,1,WDNSO-1
7,84.556,2018-08-01,1,BRBEO-1
8,48.326,2018-08-01,1,RCBKO-2
9,645.188,2018-08-01,1,DRAXX-1


<br>

### Single Generator Filtering & Processing

In [7]:
## Get unit ids, add datetime column
## Go long to wide with datetime as index and unit ids as columns with quantity as the values

In [8]:
def add_df_datetime(df, SP_col, date_col): 
    # Combining date and SP column into a datetime one
    df = df.copy()
    df['time_delta'] = pd.to_timedelta(df[SP_col].astype(int)*30 - 30, unit='m')
    df['local_datetime'] = pd.to_datetime(df[date_col]) + df['time_delta']
    df = df.drop(columns=[SP_col, date_col, 'time_delta'])
    return df

def localize_datetime(df, datetime_col):
    df = df.set_index(datetime_col, drop=True)
    s_UTC_index = df.index.tz_localize('Europe/London')  
    df['UTC'] = s_UTC_index.tz_convert('UTC')    
    return df.drop(columns=['UTC']), pd.Series(df['UTC'], index=df.index)

def proccess_df(df, SP_col, date_col, datetime_col, id_col, quantity_col):
    df = add_df_datetime(df, SP_col, date_col)
    df, s_UTC = localize_datetime(df, datetime_col)
    df = df.reset_index().pivot(datetime_col, id_col, quantity_col)
    df = df.replace(np.NaN, 0)
    df = df.apply(pd.to_numeric)
    return df, s_UTC

In [9]:
## Generator IDs for filtering
drax_main_units = ['DRAXX-1', 'DRAXX-2', 'DRAXX-3', 'DRAXX-4', 'DRAXX-6']
drax_all_units = drax_main_units + ['DRAXD-1', 'DRAXD-2', 'DRAXX-9G', 'DRAXX-10G', 'DRAXX-12G']

In [10]:
## Dataframe filtering and processing
df_drax_filt = df_gen_trim[df_gen_trim.nGCBMUnitID.isin(drax_main_units)] # Filtering
df_drax_proc, s_UTC = proccess_df(df_drax_filt, 'settlementPeriod', 'settlementDate', 'local_datetime', 'nGCBMUnitID', 'quantity')# Processing
df_drax_proc.head(10)

nGCBMUnitID,DRAXX-1,DRAXX-2,DRAXX-3,DRAXX-6
local_datetime,,,,
2018-08-01 00:00:00,645.188,314.768,161.332,0.0
2018-08-01 00:30:00,645.136,300.496,0.000,0.0
2018-08-01 01:00:00,645.188,300.180,0.000,0.0
2018-08-01 01:30:00,645.224,300.280,0.000,0.0
2018-08-01 02:00:00,645.140,300.896,0.000,0.0
2018-08-01 02:30:00,645.156,300.272,0.000,0.0
2018-08-01 03:00:00,645.340,309.652,0.000,0.0
2018-08-01 03:30:00,618.816,300.232,0.000,0.0
2018-08-01 04:00:00,587.688,300.212,0.000,0.0
